# Семинар № 5 - Распознавание объектов

# 1.EDA

### Установите и импортируйте необходимые библиотеки

In [ ]:
# !pip install albumentations
# !pip install opencv-contrib-python
# !pip install "opencv-python-headless<4.3"

In [ ]:
import os
import re
import random

import cv2
import numpy as np 
import pandas as pd 
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

import torch
import torchvision
from torchvision import transforms 
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

In [ ]:
def set_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

In [ ]:
set_seed(42)

## [Загрузить](https://disk.yandex.ru/d/zwlQ0xbBygL58Q) обучающий и тестовый файл

In [ ]:
train_df = pd.read_csv("global-wheat-detection/train.csv")
submit = pd.read_csv("global-wheat-detection/sample_submission.csv")

In [ ]:
train_df.head()

In [ ]:
# Удалите ненужные столбцы
train_df = train_df.drop(columns=['width','height','source']) 

In [ ]:
# В обучающем наборе данных всего 3373 уникальных изображения
train_df['image_id'].nunique() 

In [ ]:
# максимальное количество полей в одном изображении - 116
(train_df['image_id'].value_counts()).max()  

In [ ]:
# Минимальное количество блоков в одном изображении равно 1
(train_df['image_id'].value_counts()).min() 

### Разделение размера блока в формате [xmin, ymin, w, h]
#### Позже мы преобразуем определение box в [xmin, ymin, xmax, ymax]

In [ ]:
train_df['x'] = -1
train_df['y'] = -1
train_df['w'] = -1
train_df['h'] = -1

def expand_bbox(x):
    r = np.array(re.findall("([0-9]+[.]?[0-9]*)", x))
    if len(r) == 0:
        r = [-1, -1, -1, -1]
    return r

In [ ]:
train_df[['x', 'y', 'w', 'h']] = np.stack(train_df['bbox'].apply(lambda x: expand_bbox(x))) ##Lets convert the Box in 
train_df['x'] = train_df['x'].astype(np.float32)                                        #in our desired formate    
train_df['y'] = train_df['y'].astype(np.float32)
train_df['w'] = train_df['w'].astype(np.float32)
train_df['h'] = train_df['h'].astype(np.float32)

In [ ]:
train_df.head() 

In [ ]:
submit[['x', 'y', 'w', 'h']] = np.stack(submit['PredictionString'].apply(lambda x: [0, 0, 1, 1]))

In [ ]:
submit.head()

### Разделение данных на обучающий и валидационный наборы

In [ ]:
image_ids = train_df['image_id'].unique()
valid_ids = image_ids[-665:]
train_ids = image_ids[:-665]

valid_df = train_df[train_df['image_id'].isin(valid_ids)]
train_df = train_df[train_df['image_id'].isin(train_ids)]

# 2.Написание пользовательского набора данных для нашей работы

### 2.1 Написание пользовательского набора данных для обучающих и валидационных изображений

In [ ]:
class WheatDataset(Dataset):
    def __init__(self, dataframe, image_dir, transforms=None,train=True):
        super().__init__()

        self.image_ids = dataframe['image_id'].unique()
        self.df = dataframe
        self.image_dir = image_dir
        self.transforms = transforms
        self.train = train

    def __len__(self) -> int:
        return self.image_ids.shape[0]

    def __getitem__(self, index: int):

        image_id = self.image_ids[index]
        # чтение изображения
        image = None
        
        # чтение данных (bbox) - x1, y1, x2, y2
        records = self.df[self.df['image_id'] == image_id]   
        boxes = None

        # чтение класса
        labels = None
        
        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        target['image_id'] = torch.tensor([index])

        # преобразование
        if self.transforms:
            None
            
        return image, target, image_id

In [ ]:
def get_train_transforms():
    return A.Compose(
        [
            A.RandomSizedCrop(min_max_height=(800, 800), height=1024, width=1024, p=0.5),
            A.OneOf([
                A.MotionBlur(),
                A.GaussNoise(var_limit=(0, 0.1)),
            ], p=0.7),
            A.OneOf([
                A.Transpose(),
                A.RandomRotate90(),
                A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1,
                                   rotate_limit=45,
                                   border_mode=cv2.BORDER_CONSTANT, value=0),
                A.NoOp()
            ], p=0.8),
            A.OneOf([
                A.RandomBrightnessContrast(brightness_limit=0.2,
                                           contrast_limit=0.2, p=0.8),
                A.RandomGamma(gamma_limit=(70, 130)),
                A.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2,
                                     val_shift_limit=0.2, p=0.6),
                A.NoOp()
            ], p=0.8),
            A.ToGray(p=0.01),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.Resize(height=512, width=512, p=1),
            A.Cutout(num_holes=12, max_h_size=32, max_w_size=32, fill_value=0, p=0.5),
            ToTensorV2(p=1.0),
        ],
        p=1.0,
        bbox_params=A.BboxParams(
            format='pascal_voc',
            min_area=0,
            min_visibility=0,
            label_fields=['labels']
        )
    )


def get_valid_transforms():
    return A.Compose(
        [
            A.Resize(height=512, width=512, p=1.0),
            ToTensorV2(p=1.0),
        ],
        p=1.0,
        bbox_params=A.BboxParams(
            format='pascal_voc',
            min_area=0,
            min_visibility=0,
            label_fields=['labels']
        )
    )

In [ ]:
train_dir = 'global-wheat-detection/train'
test_dir = 'global-wheat-detection/test'

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

train_dataset = WheatDataset(train_df, train_dir, get_train_transforms(), True)
valid_dataset = WheatDataset(valid_df, train_dir, get_valid_transforms(), True)


# split the dataset in train and test set
indices = torch.randperm(len(train_dataset)).tolist()

train_data_loader = DataLoader(
    train_dataset,
    batch_size=4,
    shuffle=False,
    num_workers=0,
    collate_fn=collate_fn
)

valid_data_loader = DataLoader(
    valid_dataset,
    batch_size=4,
    shuffle=False,
    num_workers=0,
    collate_fn=collate_fn
)

### Давайте визуализируем некоторые изображения с помощью ограничивающей рамки

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
images, targets, image_ids = next(iter(train_data_loader))
images = list(image.to(device) for image in images)
targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

idx = 2
boxes = targets[idx]['boxes'].cpu().numpy().astype(int)
sample = images[idx].permute(1,2,0).cpu().numpy()

fig, ax = plt.subplots(1, 1, figsize=(16, 8))

for box in boxes:
    sample = cv2.rectangle(sample.copy(), (box[0], box[1]), (box[2], box[3]), (220, 0, 0), 3)
    
ax.set_axis_off()
ax.imshow(sample)

# 3.Точная настройка модели

### Определение модели

Faster R-CNN - это модель, которая предсказывает как ограничительные рамки, так и оценки классов для потенциальных объектов на изображении.


Давайте объясним, как работает эта архитектура, поскольку RCNN состоит из 3 частей

1. Часть 1: Слои свертки: Архитектура CNN формируется стеком отдельных слоев, которые преобразуют входной объем в выходной объем (например, хранящий оценки класса) с помощью дифференцируемой функции.Сверточные сети были вдохновлены биологическими процессами в том смысле, что структура связей между нейронами напоминает организацию зрительной коры головного мозга животных. Отдельные нейроны коры головного мозга реагируют на стимулы только в ограниченной области поля зрения, известной как рецептивное поле. Рецептивные поля разных нейронов частично перекрываются таким образом, что они охватывают все поле зрения.

2. Часть 2: Сеть предложения региона (RPN): RPN - это небольшая нейронная сеть, скользящая по последней карте объектов слоев свертки и предсказывающая, есть объект или нет, а также предсказывающая ограничивающую рамку этих объектов.

3. Часть 3: Предсказание классов и ограничивающих рамок: Теперь мы используем другую полностью связанную нейронную сеть, которая принимает в качестве inpt области, предложенные RPN, и предсказывает класс объекта (классификация) и ограничивающие рамки (регрессия).

In [ ]:
# загрузить модель; предварительно обученную на COCO
model = torchvision.models.detection.fasterrcnn_mobilenet_v3_large_fpn(pretrained=True, pretrained_backbone=True)

In [ ]:
# класс № 1 (пшеница) + фон
num_classes = 2  

# получить количество входных объектов для классификатора
in_features = model.roi_heads.box_predictor.cls_score.in_features

# замените предварительно подготовленную головку на новую
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

In [ ]:
class Averager:      ##Return the average loss 
    def __init__(self):
        self.current_total = 0.0
        self.iterations = 0.0

    def send(self, value):
        self.current_total += value
        self.iterations += 1

    @property
    def value(self):
        if self.iterations == 0:
            return 0
        else:
            return 1.0 * self.current_total / self.iterations

    def reset(self):
        self.current_total = 0.0
        self.iterations = 0.0

### Давайте потренируем нашу модель

In [ ]:
device = 'cpu'

In [ ]:
model.train()
model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.01, momentum=0.9, weight_decay=1e-4)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.5)

num_epochs = 1

loss_hist = Averager()
itr = 1

In [ ]:
for epoch in range(num_epochs):
    loss_hist.reset()
    
    for images, targets, image_ids in tqdm(train_data_loader):
        
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)

        losses = sum(loss for loss in loss_dict.values()).type(torch.float32)
        loss_value = losses.item()

        loss_hist.send(loss_value)

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        if itr % 50 == 0:
            print(f"Iteration #{itr} loss: {loss_value}")

        itr += 1
    
    if lr_scheduler is not None:
        lr_scheduler.step()

    print(f"Epoch #{epoch} loss: {loss_hist.value}")

# 4. Предсказание

### Давайте загрузим тестовые данные

In [ ]:
test_dataset = WheatDataset(submit, test_dir, get_valid_transforms(), False)

In [ ]:
test_data_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

### Установите пороговое значение для прогнозирования ограничивающей рамки

In [ ]:
detection_threshold = 0.45

In [ ]:
results = []
model.eval()

for images, _, image_ids in tqdm(test_data_loader):    

    images = list(image.to(device) for image in images)
    outputs = model(images)

    for i, image in enumerate(images):

        boxes = outputs[i]['boxes'].data.cpu().numpy()    ##Formate of the output's box is [Xmin,Ymin,Xmax,Ymax]
        scores = outputs[i]['scores'].data.cpu().numpy()
        
        boxes = boxes[scores >= detection_threshold].astype(np.int32) #Compare the score of output with the threshold and
        scores = scores[scores >= detection_threshold]                    #slelect only those boxes whose score is greater
                                                                          # than threshold value
        image_id = image_ids[i]
        
        boxes[:, 2] = boxes[:, 2] - boxes[:, 0]         
        boxes[:, 3] = boxes[:, 3] - boxes[:, 1]         #Convert the box formate to [Xmin,Ymin,W,H]

In [ ]:
sample = images[1].permute(1, 2, 0).cpu().numpy()
boxes = outputs[1]['boxes'].data.cpu().numpy()
scores = outputs[1]['scores'].data.cpu().numpy()

boxes = boxes[scores >= detection_threshold].astype(np.int32)

### Давайте построим некоторые из наших прогнозов

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16, 8))

for box in boxes:
    sample = cv2.rectangle(sample.copy(),
                  (box[0], box[1]),
                  (box[2], box[3]),
                  (220, 0, 0), 2)
    
ax.set_axis_off()
ax.imshow(sample)

## Bonus - AP

Расчет метрики для задачи распознавания

In [ ]:
def compute_overlap(a: np.array, b: np.array) -> np.array:
    """
    Args
        a: (N, 4) ndarray of float [xmin, ymin, xmax, ymax]
        b: (K, 4) ndarray of float [xmin, ymin, xmax, ymax]

    Returns
        overlaps: (N, K) ndarray of overlap between boxes a and boxes b
    """
    a_area = (a[:, 2] - a[:, 0]) * (a[:, 3] - a[:, 1])
    b_area = (b[:, 2] - b[:, 0]) * (b[:, 3] - b[:, 1])

    dx = np.minimum(np.expand_dims(a[:, 2], axis=1), b[:, 2]) - np.maximum(np.expand_dims(a[:, 0], axis=1), b[:, 0])
    dy = np.minimum(np.expand_dims(a[:, 3], axis=1), b[:, 3]) - np.maximum(np.expand_dims(a[:, 1], axis=1), b[:, 1])

    intersection = np.maximum(dx, 0) * np.maximum(dy, 0)
    union = np.expand_dims(a_area, axis=1) + b_area - intersection
    overlaps = intersection / union

    return overlaps


def compute_ap(recall, precision):
    """ Compute the average precision, given the recall and precision curves.
    Code originally from https://github.com/rbgirshick/py-faster-rcnn.
    # Arguments
        recall:    The recall curve (list).
        precision: The precision curve (list).
    # Returns
        The average precision as computed in py-faster-rcnn.
    """
    # correct AP calculation
    # first append sentinel values at the end
    mrec = np.concatenate(([0.], recall, [1.]))
    mpre = np.concatenate(([0.], precision, [0.]))

    # compute the precision envelope
    for i in range(mpre.size - 1, 0, -1):
        mpre[i - 1] = np.maximum(mpre[i - 1], mpre[i])

    # to calculate area under PR curve, look for points
    # where X axis (recall) changes value
    i = np.where(mrec[1:] != mrec[:-1])[0]

    # and sum (\Delta recall) * prec
    ap = np.sum((mrec[i + 1] - mrec[i]) * mpre[i + 1])
    return ap

In [ ]:
def evaluate_res(inference_res, iou_threshold=0.5, score_threshold=0.05):
    """ Evaluate a given dataset using a given model.
    # Arguments
        inference_res   : inference results for whole imageset List((target,prediction)),
            where targets {'boxes':np.array[4,n], 'labels':np.array[n]},
            prediction {'boxes':np.array[4,n], 'labels':np.array[n], scores: np.array[n]}
            example:

            [({'boxes': np.array([[1321.8750,  274.6667, 1348.8750,  312.6667]]),
                'labels': np.array([1])},
              {'boxes': np.array([[1323.5446,  275.2711, 1350.2203,  315.9069],
                        [ 119.2671, 1227.5459,  171.1528, 1277.9830],
                        [ 240.5078, 1147.3656,  270.7879, 1205.0126],
                        [ 140.9097, 1231.9814,  173.9967, 1285.4724]]),
                'scores': np.array([0.9568, 0.3488, 0.1418, 0.0771]),
                'labels': np.array([1, 1, 1, 1])}),
             ({'boxes': np.array([[ 798.7500, 1357.3334,  837.7500, 1396.6666],
                        [ 829.1250,  777.3333,  873.3750,  818.0000],
                        [ 886.5000,   34.6667,  916.5000,   77.3333]]),
                'labels': np.array([1, 1, 1])},
              {'boxes': np.array([[ 796.5808, 1354.9255,  836.5349, 1395.8972],
                        [ 828.8597,  777.9426,  872.5923,  819.8660],
                        [ 887.7839,   37.1435,  914.8092,   76.3933]]),
                'scores': np.array([0.9452, 0.8701, 0.8424]),
                'labels': np.array([1, 1, 1])})]

        iou_threshold   : The threshold used to consider when a detection is positive or negative.
        score_threshold : The score confidence threshold to use for detections.
    """
    false_positives = np.zeros((0,))
    true_positives = np.zeros((0,))
    scores = np.zeros((0,))
    num_annotations = sum([t['labels'].shape[0] for t, _ in inference_res])

    for i, (annotations, detections) in enumerate(inference_res):
        # detections = p
        # annotations = t
        detected_annotations = []

        if annotations['labels'].shape[0] == 0:  # no objects was there
            false_positives = np.append(false_positives, np.ones(detections['labels'].shape[0]))
            true_positives = np.append(true_positives, np.zeros(detections['labels'].shape[0]))
            continue

        for d in np.arange(detections['labels'].shape[0]):
            if detections['scores'][d] > score_threshold:
                scores = np.append(scores, detections['scores'][d])

                overlaps = compute_overlap(np.expand_dims(detections['boxes'][d].astype(np.double), axis=0),
                                           annotations['boxes'].astype(np.double))
                assigned_annotation = np.argmax(overlaps, axis=1)
                max_overlap = overlaps[0, assigned_annotation][0]

                true_label = annotations['labels'][assigned_annotation][0]
                predict_label = detections['labels'][d]
                if max_overlap >= iou_threshold and assigned_annotation not in detected_annotations and true_label == predict_label:
                    false_positives = np.append(false_positives, 0)
                    true_positives = np.append(true_positives, 1)
                    detected_annotations.append(assigned_annotation)
                else:
                    false_positives = np.append(false_positives, 1)
                    true_positives = np.append(true_positives, 0)

    # F1@IoU
    plain_recall = np.sum(true_positives) / np.fmax(num_annotations, np.finfo(np.float64).eps)
    plain_precision = np.sum(true_positives) / np.fmax(np.sum(true_positives) + np.sum(false_positives),
                                                       np.finfo(np.float64).eps)
    F1 = 2 * plain_precision * plain_recall / np.fmax(plain_precision + plain_recall,
                                                      np.finfo(np.float64).eps)

    # compute false positives and true positives
    indices = np.argsort(scores)[::-1]
    false_positives = np.cumsum(false_positives[indices])
    true_positives = np.cumsum(true_positives[indices])
    # compute recall and precision
    recall = true_positives / num_annotations
    precision = true_positives / np.fmax(true_positives + false_positives, np.finfo(np.float64).eps)

    # compute average precision
    average_precision = compute_ap(recall, precision)

    return average_precision, F1